In [8]:
import pandas as pd

def clean_column(column_name, filename):      
    df = pd.read_csv(filename, sep=";")
    if column_name in df.columns:
        df[column_name] = df[column_name].str.replace(r'[^0-9.]', '', regex=True)
    df.to_csv(filename, index = False)

    
clean_column("Schadstoffklasse", "D:/Ibo/SFP/SFP_Notebook/car_data_csv.csv")

In [9]:
clean_column("CO₂-Emissionen2","D:/Ibo/SFP/SFP_Notebook/car_data_csv.csv")